In [23]:
import cv2
import numpy as np
import random

# Data Ingestion

We can read all of our files, and get the number of frames in each one. When reading them as tensors we will truncate to the one with the smallest number of frames. I strived to take videos of the same length (~11s), but small discrepancies are bound to exist. For our particular application, truncation ought not to matter too much.

In [24]:
# Create VideoCapture objects
parking_lot = cv2.VideoCapture('parking_lot.MOV')
patio = cv2.VideoCapture('patio.MOV')

# Get number of frames in each video
parking_lot_frames = int(parking_lot.get(cv2.CAP_PROP_FRAME_COUNT))
patio_frames = int(patio.get(cv2.CAP_PROP_FRAME_COUNT))

parking_lot_frames, patio_frames

(321, 328)

In [25]:
# Get dimensions of each frame
parking_lot_height = int(parking_lot.get(cv2.CAP_PROP_FRAME_HEIGHT))
parking_lot_width = int(parking_lot.get(cv2.CAP_PROP_FRAME_WIDTH))
patio_height = int(patio.get(cv2.CAP_PROP_FRAME_HEIGHT))
patio_width = int(patio.get(cv2.CAP_PROP_FRAME_WIDTH))

print(parking_lot_height, parking_lot_width)
print(patio_height, patio_width)

1080 1920
1080 1920


To speed up the process, we only read in 100 frames from each image.

In [27]:
# Get the random frames for each image
random.seed(42)
random_frames = []
for i in range(100):
    random_frames.append(random.randrange(0, 320)) 

Based on the number of frames and the dimensions of the frames, we will need a 4D tensor (1080x1920x3x321) to hold these videos:
- 1080x1920 for the height and width of the images
- 3 for the RGB color channels
- 100 for the frames of the images

In [28]:
# Create desired arrays
parking_lot_array = np.empty([1080, 1920, 3, 100])
patio_array = np.empty([1080, 1920, 3, 100])

# Keep track of the frame
parking_lot_frame_nb = 0
patio_frame_nb = 0
index_parking_lot = 0
index_patio = 0

# Convert the parking lot video
while parking_lot.isOpened() and parking_lot_frame_nb < parking_lot_frames:
    ret, frame = parking_lot.read()
    if parking_lot_frame_nb in random_frames:
        parking_lot_array[:,:,:,index_parking_lot] = frame
        index_parking_lot += 1
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        next
    parking_lot_frame_nb += 1
    
# Convert the patio video
while patio.isOpened() and patio_frame_nb < parking_lot_frames:
    ret, frame = patio.read()
    if patio_frame_nb in random_frames:
        patio_array[:,:,:,index_patio] = frame
        index_patio += 1
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        next
    patio_frame_nb += 1

Release the capture now that we don't need it anymore.

In [31]:
parking_lot.release()
patio.release()
cv2.destroyAllWindows()